In [1]:
import torch
import torch.nn as nn
import string
import random
import re
import time, math

In [40]:
num_epochs = 500
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [3]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [4]:
file = open('shakespeare.txt',encoding='utf8').read()
file_len = len(file)
print('file_len = ',file_len)

file_len =  1115394


In [5]:
def random_chunk():
    start_index = random.randint(0,file_len-chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor("ABCdef"))

tensor([36, 37, 38, 13, 14, 15])


In [6]:
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp,target

In [22]:
class RNN(nn.Module):
    def __init__(self,input_size,embedding_size,hidden_size,output_size,num_layers=1):
        super(RNN,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(input_size,embedding_size)
        self.rnn = nn.RNN(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size,output_size)
        
    def forward(self,input,hidden):
        out = self.encoder(input.view(1,-1)) # row vector
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden
    
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden

In [23]:
model = RNN(input_size=n_characters,embedding_size=embedding_size,hidden_size=hidden_size,
           output_size = n_characters,num_layers=2)

In [33]:
inp = char_tensor("A")
hidden = model.init_hidden()
out,hidden = model(inp,hidden)
optimizer = torch.optim.Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()

In [39]:
def test():
    start_str = "c"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 만약 max 값만 쓰는 경우에
        # 생성되는 텍스트가 다 the the the the the 이런식으로 나오기 때문입니다.
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 램덤하게 다음 글자를 뽑아내는 방식으로 자연스러운 텍스트를 생성해냅니다.
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)
    

In [41]:
for i in range(num_epochs):
    
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()
    
    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)
    loss.backward()
    optimizer.step()
    if i%50 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([2.3882], grad_fn=<DivBackward0>) 

c's to hir on at band or mang with nof ankef
Moun bund.

HIRI?NURTINNAr:
And I an math er keryithiny,

Krell bad difeat fellised deat ir orint minge !fant thane thar.

WERNCEgERCCOOOY:
An bite ! lorse 

 tensor([2.2895], grad_fn=<DivBackward0>) 

cErid fsacle thou ghen haisgiel I sheressenr ihes the soy oule on thol and heswert hecond hit eet, wordans the in, cars foanbroy tho sall mefle with wiwh
WoRrM, herat you had thed ther allit: the trowl

 tensor([2.1671], grad_fn=<DivBackward0>) 

cIED:
A shary thele kererth's hime her in I jather hore then lum this the hacrritt,
4o the bothoue gond hie seitf anes hiig fur bllave the soredin;
And sares welte.

FETS:
Oe and inds oue theur nos Com

 tensor([2.1471], grad_fn=<DivBackward0>) 

chad, pould tones houbs in mong, wors bur?

LAOMUDNGUS:
Thy rowaver the me wathas maded, aning the hean?

IUS:
Hor wnokes.

SIMELIA:
Sok lond wichaf, in houd hi's grtengove sir the gint; thand Vporlr h

 tensor([2.

In [38]:
torch.cuda.device_count()

0